# Modelagem

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

import imblearn
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, plot_confusion_matrix, classification_report

from sklearn.ensemble import IsolationForest
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

## Lendo os Dados

In [ ]:
train = pd.read_csv('/kaggle/input/new-trainnew-test/new_train.csv')
test = pd.read_csv('/kaggle/input/new-trainnew-test/new_test.csv')
train.shape

## Removendo Outliers

In [ ]:
fig, ax = plt.subplots(1, 5, figsize=(12, 4))
ax = ax.reshape(1, 5)[0]
fig.suptitle('Boxplot para Visualização de Outliers', fontsize=16)

cols_float = []
for col in train.columns:
    if train[col].dtype == float and not col.startswith('SQB'):
        cols_float.append(col)

for i, col in enumerate(cols_float):
    ax[i].boxplot(train[col])
    ax[i].set_title(col, fontsize=16)

plt.tight_layout()

In [ ]:
clf = IsolationForest(max_samples=300, random_state=42)
clf.fit(train[cols_float])
train['outliers'] = clf.predict(train[cols_float])
print('Outliers:', len(train.query('outliers < 0')))

In [ ]:
train = train.query('outliers > 0') # Filtrando somente dados que não são outliers
train.drop(columns=['outliers'], inplace=True)

- Fazendo a remoção de outliers com IsolationForest.

- O modelo IsolatioForest funciona da seguinte maneira: São definidos grupos de dados aleatoriamente dentro do conjunto de dados utilizado no treinamento. Cada conjunto de dados será representado por uma árvore isolada (IsolationTree ou iTree). Cada iTree pontua os dados com uma pontuação de anomalia, e ao final é tirado a média destas pontuações de anomalia. O valor da média da pontuação de anomalia é utilizado pelo modelo para encontar os pontos mais distantes da base de dados, esses pontos distantes são os outliers.

## Preparando os Dados

In [ ]:
def change_dependency_yes(row):
    if row.dependency == 'yes':
        return (row.hogar_nin + row.hogar_mayor) / row.hogar_adul
    return row.dependency

In [ ]:
train['edjefe'] = train['edjefe'].replace({'yes': '1', 'no': '0'}).astype(int)
train['edjefa'] = train['edjefe'].replace({'yes': '1', 'no': '0'}).astype(int)
train['dependency'] = train.apply(change_dependency_yes, axis=1)
train['dependency'] = train['dependency'].replace({'no': '0'}).astype(float)

In [ ]:
test['edjefe'] = test['edjefe'].replace({'yes': '1', 'no': '0'}).astype(int)
test['edjefa'] = test['edjefe'].replace({'yes': '1', 'no': '0'}).astype(int)
test['dependency'] = test.apply(change_dependency_yes, axis=1)
test['dependency'] = test['dependency'].replace({'no': '0'}).astype(float)

In [ ]:
X_train_all = train.drop(columns=['Target', 'Id', 'idhogar'])
y_train_all = train['Target']

X_train, X_valid, y_train, y_valid = train_test_split(X_train_all, y_train_all, test_size=0.15, random_state=42)

## Tratando Classes Desbalanceadas

In [ ]:
X_train_all.shape, y_train_all.shape

In [ ]:
# Oversampling
sm = SMOTE()
X_train_sm, y_train_sm = sm.fit_resample(X_train, y_train)
y_train_sm.value_counts()

- Fazendo oversampling com método SMOTE. Este método criará novas observações baseado no padrão de comportamento dos dados. Ao final serão criados novos registros até que todas as classes estejam balanceadas.

In [ ]:
X_train_sm.shape

In [ ]:
# Undesampling
tl = TomekLinks()
X_train_tl, y_train_tl = tl.fit_resample(X_train_sm, y_train_sm)
X_train_tl.shape, y_train_tl.shape

- Após o oversampling com o método SMOTE foi utilizado o método TomekLinks para que seja removido as observações semelhantes (Undersampling). Um dos motivos de utilizar o TomekLinks após o uso do SMOTE é para refinar as observações, retirando os registros semelhantes que possam "enviesar" o modelo.

In [ ]:
y_train_tl.value_counts()

## Treinando e Avalidando os Modelos

### Random Forest

In [ ]:
param_grid = {
    'n_estimators': [200, 400],
    'oob_score': [True],
    'min_samples_leaf': [1, 2],
    'random_state': [42]
}
rfg = GridSearchCV(RandomForestClassifier(), param_grid=param_grid, cv=10, n_jobs=-1, return_train_score=True, scoring='accuracy')
rfg.fit(X_train_tl, y_train_tl)

In [ ]:
grid_df = pd.DataFrame(rfg.cv_results_)
rank = grid_df[['mean_train_score','mean_test_score','rank_test_score']].sort_values('rank_test_score').head(3)
BEST_PARAMS = grid_df.sort_values('rank_test_score').params.to_list()[0]
rank

In [ ]:
plt.plot(grid_df['mean_train_score'],label='train')
plt.plot(grid_df['mean_test_score'],label='test')
plt.legend()
plt.show()

- O gráfico acima apresenta o comportamento do desemepenho de treino e teste realizado no CrossValidation do GridSearch. Podemos notar um alto desempenho no treino e um desempenho inferior nos testes, devido a diferença de valores não ser tão distante (1,00 - treino e 0,935 - teste), este gráfico mostra que o modelo não está "overfittado".

In [ ]:
y_valid.value_counts()

In [ ]:
plot_confusion_matrix(
   rfg, X_valid, y_valid, cmap=plt.cm.Blues,values_format='.4g'
)
plt.show()
print(classification_report(y_valid, rfg.predict(X_valid)))

- Analisando gráfico acima e a matriz de confusão podemos notar que o modelo de RandomForest obteve uma acurácia de 92%. Outro ponto importante a ser observado é o *precision* e *recall*:
- Para a classe 4 foi obtido uma precisão de 94%.
- Já o recall da classe 4 foi igual a 98%.
- O f1-score para classe 4 = 96%.


- Foram citados as medidas da classe 4 pelo fato desta classe possuir a maior ocorrência na base de dados.


- **Precision**: De todos os valores previstos pelo modelo como classe X , quantos valores o modelo preveu como classe X?
- **Recall**: De todos os valores que realmente são da classe X quantos valores o modelo preveu sendo da classe X?
- **f1-Score**: Média Harmônica - Medida única para repersentar o recall e precision juntos.

### GBM

- Observação: Devido ao tempo de execução, não será utilizado o GridSearch para os modelos de boost.

In [ ]:
gbm = GradientBoostingClassifier(n_estimators=400)
gbm.fit(X_train_tl, y_train_tl)

In [ ]:
plot_confusion_matrix(
   gbm, X_valid, y_valid, cmap=plt.cm.Blues,values_format='.4g'
)
plt.show()
print(classification_report(y_valid, gbm.predict(X_valid)))

Analisando gráfico acima e a matriz de confusão podemos notar que o modelo de GradientBoost obteve uma acurácia geral de 80%.
- Para a classe 4 foi obtido uma precisão de 84%.
- Já o recall da classe 4 foi igual a 94%.
- O f1-score para classe 4 foi igual 89%.

### AdaBoost

In [ ]:
ada = AdaBoostClassifier(n_estimators=400)
ada.fit(X_train_tl, y_train_tl)

In [ ]:
plot_confusion_matrix(
   ada, X_valid, y_valid, cmap=plt.cm.Blues,values_format='.4g'
)
plt.show()
print(classification_report(y_valid, ada.predict(X_valid)))

Analisando gráfico acima e a matriz de confusão podemos notar que o modelo de AdaBoost obteve uma acurácia de 66%.
- Para a classe 4 foi obtido uma precisão de 78%.
- O recall da classe 4 foi igual a 86%.
- O f1-score para classe 4 igual a 82%.

### XGBoost

In [ ]:
xgb = XGBClassifier(n_estimators=400, eval_metric='mlogloss')
xgb.fit(X_train_tl, y_train_tl)

In [ ]:
plot_confusion_matrix(
   xgb, X_valid, y_valid, cmap=plt.cm.Blues,values_format='.4g'
)
plt.show()
print(classification_report(y_valid, xgb.predict(X_valid)))

Analisando gráfico acima e a matriz de confusão podemos notar que o modelo de XGBoost obteve uma acurácia de 90%.
- Para a classe 4 foi obtido uma precisão de 92%.
- O recall da classe 4 foi igual a 97%.
- O f1-score para classe 4 foi igual a 95%.

### CatBoost

In [ ]:
cbc = CatBoostClassifier(random_state=42, n_estimators=400, verbose=False)
cbc.fit(X_train_tl, y_train_tl)

In [ ]:
plot_confusion_matrix(
   cbc, X_valid, y_valid, cmap=plt.cm.Blues,values_format='.4g'
)
plt.show()
print(classification_report(y_valid, cbc.predict(X_valid)))

Analisando gráfico acima e a matriz de confusão podemos notar que o modelo de CatBoost obteve uma acurácia de 86%.
- Para a classe 4 foi obtido uma precisão de 89%.
- O recall da classe 4 foi igual a 97%.
- O f1-score para classe 4 foi igual a 92%.

### LightGBM

In [ ]:
lgbm = LGBMClassifier(random_state=42, n_estimators=400)
lgbm.fit(X_train_tl, y_train_tl)

In [ ]:
plot_confusion_matrix(
   lgbm, X_valid, y_valid, cmap=plt.cm.Blues,values_format='.4g'
)
plt.show()
print(classification_report(y_valid, lgbm.predict(X_valid)))

Analisando gráfico acima e a matriz de confusão podemos notar que o modelo de LightGBM obteve uma acurácia de 90%.
- Para a classe 4 foi obtido uma precisão de 92%.
- O recall da classe 4 foi igual a 98%.
- O f1-score para classe 4 foi igual a 95%.

### Resumo
- Dentre os Modelos Treinados o melhor modelo foi o RandomForest. Os fatores de escolha foram: Acurárcia, Precision e F1-Score (em especial da classe 4).
        Acc, Prec (classe 4), F1 (Classe 4)
- **RF**: 92%, 94%, 96%
- **Light**: 90%, 92%, 95%
- **XGB**: 90%, 92%, 95%
- **CAT**: 86%, 89%, 92%
- **GBM**: 79%, 84%, 89%
- **ADA**: 66%, 78%, 82%



- A Explicação sobre **Precision**, **Recall** e **F1-Score** estão na célula de explicação do RandomForest.


- Existe a possibilidade do LightGBM ou XGBoost performarem ainda melhor caso sejam utilizados sob o GridSearch com diferentes configurações de parâmetros, porém, o GridSearch não foi utilizado devido a demora no tempo e treinamento.


## Aplicando O RandomForest Melhorado na Base de Teste


### Ajustando os Dados Finais

In [ ]:
tl = TomekLinks()
sm = SMOTE()

X_train_sm_all, y_train_sm_all = sm.fit_resample(X_train_all, y_train_all) # Oversampling
X_train_tl_all, y_train_tl_all = tl.fit_resample(X_train_sm_all, y_train_sm_all) # Undersampling

### Treinando o Melhor Modelo

In [ ]:
model = RandomForestClassifier(**BEST_PARAMS)
model.fit(X_train_tl_all, y_train_tl_all)

### Predições

In [ ]:
submission = test[['Id']].copy()
y_pred = model.predict(test.drop(columns=['Id', 'idhogar']))
submission['Target'] = y_pred

submission.to_csv('submission.csv', index=False)